## Support Vector Machines
* Poderosa metodologia para resolver problemas de aprendizagem de máquina.
* Proposto em 1995 pelo russo Vladimir Vapnik.
* Muito utilizado atualmente em diversos tipos de aplicações.
* O problema dos problemas não linearmente separáveis 
* O problema dos problemas não linearmente separáveis


![](https://i.ibb.co/tsDTKzn/SVM-Picture1.png)

* Problema não linearmente separável

### O processo de classificação:
  + Entendendo o parâmetro C, Soft Margin: 
    - A aplicação de um método puramente linear para classificar um conjunto de dados pode sofrer com dois problemas bastante comuns:
      - Outliers
      - Exemplos rotulados erroneamente
  + Mesmo assim o SVM ainda assim pode ser aplicado através do uso do parâmetro C (soft margin - variáveis de folga)
#### O truque do Kernel:
    - Realiza a projeção das características em um espaço multidimencional, maior que o vetor de entrada onde as classes podem ser separadas por um hiperplano.

![](https://i.ibb.co/n19dmvq/SVM-Picture2.png)

#### Vantagens:
  + Considerado o estado da arte dos classificadores.
  + Trabalha com problemas linearmente e não-linearmente separáveis.
  + Desempenho superior a maioria dos classificadores tradicionais.
  
#### Desvantagens:
  + A calibragem dos parâmetros envolvidos e escolha do tipo de kernel não é uma tarefa trivial.
  + O tempo de treinamento pode ser extremamente lento e computacionalmente complexo dependendo da quantidade de características.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt 

from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

# Importe as bibliotecas de Pipelines e Pré-processadores
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Importa o pacote OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

import sys

## 1) Importando os dados já tratados sem OneHotEncoding:

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/datasets"
# data = pd.read_csv('/content/drive/My Drive/datasets/dados_clust_v2.csv',sep=',') # faz a leitura dos dados

data = pd.read_csv("/Users/andrerodrigues/Google Drive/FIAP-MBA 8IA/006 - Modelos de IA e ML/Trabalho_Final_2/Exercicio8_v2/dados_clust_v2.csv", sep =",")

data = data.iloc[:,1:]
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,dbscan,kmeans,som
0,-122.23,452600.0,129,41,880,3,126,322,83.252,37.88,1,3,1
1,-122.22,358500.0,1106,21,7099,3,1138,2401,83.014,37.86,0,2,2
2,-122.24,352100.0,190,52,1467,3,177,496,72.574,37.85,1,3,1
3,-122.25,341300.0,235,52,1274,3,219,558,56.431,37.85,1,1,1
4,-122.25,342200.0,280,52,1627,3,259,565,38.462,37.85,1,1,1


## 2) Importando os dados já tratados com OneHotEncoding:

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/datasets"
# data_ohe = pd.read_csv('/content/drive/My Drive/datasets/data_ohe_nostd_v2.csv',sep=',') # faz a leitura dos dados

data_ohe = pd.read_csv("/Users/andrerodrigues/Google Drive/FIAP-MBA 8IA/006 - Modelos de IA e ML/Trabalho_Final_2/Exercicio8_v2/data_ohe_nostd_v2.csv", sep =",")

data_ohe = data_ohe.iloc[:,1:]
data_ohe["dbscan"] = data["dbscan"]
data_ohe["kmeans"] = data["kmeans"]
data_ohe["som"] = data["som"]
data_ohe.head()

ModuleNotFoundError: No module named 'google'

#### a) Apenas como curiosidade, vejamos como as classes estão distribuidas em 'dbscan', 'kmeans' e 'som'

In [ ]:
plt.figure(figsize=(10,7))
plot1 = sb.countplot(x='dbscan',data=data, palette='hls')
plt.title("Verificando classes de dbscan")
plt.show(plot1)
plt.figure(figsize=(10,7))
plot2 = sb.countplot(x='kmeans',data=data, palette='hls')
plt.title("Verificando classes de kmeans")
plt.show(plot2)
plt.figure(figsize=(10,7))
plot3 = sb.countplot(x='som',data=data, palette='hls')
plt.title("Verificando classes de som")
plt.show(plot3)

## 3) Seleciona o dataset para ser tratado pelo Modelo de Machine Learning e Separa as Classes

In [5]:
import traceback

def dataset_for_machine(dataset, target):
    
    # Tente receber o nome do arquivo
    try:
        d = dataset
        t = target
        
        
    
        if d == 'data':
            #No caso do DBSCAN, as amostras cujo o cluster for igual a 0, são consideradas outliers e não podemos considera-las como classe do problema.
            if t == 'dbscan':
                data.drop(data[data.dbscan == 0].index, inplace=True)
            # Troca o nome do dataset para df. A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df.
            df = data
            # Separa a classe
            classes = df[t]
            df.drop(columns=['dbscan','kmeans', 'som'], axis=1, inplace=True)


        elif d == 'data_ohe':
            #No caso do DBSCAN, as amostras cujo o cluster for igual a 0, são consideradas outliers e não podemos considera-las como classe do problema.
            if t == 'dbscan':
                data.drop(data[data.dbscan == 0].index, inplace=True)
            # Troca o nome do dataset para df. A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df.
            df = data_ohe
            # Separa a classe
            classes = df[t]
            df.drop(columns=['dbscan','kmeans', 'som'], axis=1, inplace=True)

        else:
            print("Escolha entre 'data' ou 'data_ohe'")
            exit

    # Se ocorrer um erro
    except:
        # Mostre na tela
        trace = traceback.format_exc()
        print ('Aconteceu um erro:\n', trace)
        # Encerre o programa
        raise SystemExit
        
    return df, classes
    
# df, classes = dataset_for_machine(input("Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe') e o target('kmeans' ou 'som') separados por ',':"))
dataset = input("Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe'):")
target = input("Entre com o target que será tratado pela maquina ('dbscan','kmeans' ou 'som'):")
df, classes = dataset_for_machine(dataset, target)

Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe'): data_ohe
Entre com o target que será tratado pela maquina ('dbscan','kmeans' ou 'som'): som


* A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df. A seleção se ele vai ser passado com ou sem One Hot Encoding foi feita acima.

In [6]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,-122.23,452600.0,129,41,880,3,126,322,83.252,37.88
1,-122.22,358500.0,1106,21,7099,3,1138,2401,83.014,37.86
2,-122.24,352100.0,190,52,1467,3,177,496,72.574,37.85
3,-122.25,341300.0,235,52,1274,3,219,558,56.431,37.85
4,-122.25,342200.0,280,52,1627,3,259,565,38.462,37.85


In [7]:
classes

0        11
1        23
2         2
3         1
4         1
         ..
20635     8
20636     8
20637     8
20638     8
20639     9
Name: som, Length: 20640, dtype: int64

## 4) Faz a validação via crossvalidation (k-fold)

In [8]:
def Acuracia(clf,X,y):
    resultados = cross_val_predict(clf, X, y, cv=5)
    return metrics.accuracy_score(y,resultados)

## 5) Pre-processamento de dados

In [9]:
# def remove_features(lista_features):
#     for i in lista_features:
#         data.drop(i, axis=1, inplace=True)
#     return 0

## 9) Support Vector Machines 
#### a) Preparando os pipes com as configurações do parâmetro C, Kernel e Normalização dos dados.

In [10]:
pip_1 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC())
])

pip_2 = Pipeline([
    ('min_max_scaler', MinMaxScaler()),
    ('clf', svm.SVC())
])

pip_3 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='rbf'))
])

pip_4 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='poly'))
])

pip_5 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='linear'))
])

#### b) Teste com apenas LabelEncoder na coluna 'classes' usando o pipeline 'pip_1'

In [11]:
Acuracia(pip_1,df,classes)

0.8064922480620155

#### c) Teste com apenas LabelEncoder na coluna 'classes' usando o pipeline 'pip_2'

In [12]:
Acuracia(pip_2,df,classes)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/andrerodr

0.5961240310077519

#### d) Testando Kernels 
##### Kernel rbf

In [13]:
Acuracia(pip_3,df,classes)

0.8064922480620155

##### Kernel Polynomial

In [14]:
Acuracia(pip_4,df,classes)

0.7914728682170543

##### Kernel Linear

In [15]:
Acuracia(pip_5,df,classes)

0.7749515503875969

## 9) GridSearch

In [16]:
from sklearn.model_selection import GridSearchCV

lista_C = [0.001, 0.01, 0.1, 1, 10,100]
lista_gamma = [0.001, 0.01, 0.1, 1, 10, 100]

In [17]:
parametros_grid = dict(clf__C=lista_C, clf__gamma=lista_gamma)

#### a) Faz o tuning dos parametros testando cada combinação utilziando CrossValidation com 10 folds e analisando a acurácia

In [18]:
grid = GridSearchCV(pip_3, parametros_grid, cv=10, scoring='accuracy')

In [19]:
grid.fit(df,classes)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
               

In [23]:
grid.cv_results_

{'mean_fit_time': array([11.95455461, 11.99989352, 12.64192593, 12.77619779, 13.34505432,
        16.1200505 , 12.69238286, 13.44576983,  9.67682428, 13.1132473 ,
        24.64275191, 31.66997061, 11.6857105 ,  5.87732592,  2.98271687,
         8.87172487, 32.68143523, 41.12402542,  5.75126743,  2.19478297,
         1.55494006,  7.4189306 , 40.34208546, 47.7613169 ,  2.16110573,
         1.29801109,  1.42495501,  7.36705437, 42.29419589, 51.5440706 ,
         1.35669022,  1.30913668,  1.65786629,  7.38282852, 42.19856837,
        51.61856177]),
 'std_fit_time': array([0.14642359, 0.24080167, 0.22524522, 0.51081289, 0.56643346,
        1.51778675, 0.8828497 , 0.71000304, 0.71811564, 0.2904331 ,
        0.23539769, 0.25329293, 0.09295262, 0.02540758, 0.02463476,
        0.12051416, 0.18913444, 0.32742822, 0.09529462, 0.01469355,
        0.01030928, 0.06906464, 0.15351249, 0.31568462, 0.01742822,
        0.01087162, 0.01073107, 0.08837224, 0.57316649, 0.24595884,
        0.01378405, 0.008

In [24]:
grid.best_params_

{'clf__C': 10, 'clf__gamma': 1}

In [25]:
grid.best_score_

0.8609496124031008

#### b) Métricas de Avaliação de Modelos

In [26]:
pip_6 = Pipeline([
('scaler',StandardScaler()),
('clf', svm.SVC(kernel='rbf',C=10,gamma=1))
])

In [27]:
resultados = cross_val_predict(pip_6, df, classes, cv=10)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [28]:
c_names = [str(i) for i in np.unique(classes)]
print (metrics.classification_report(classes,resultados,target_names=c_names))

              precision    recall  f1-score   support

           1       0.90      0.92      0.91       483
           2       0.85      0.86      0.86       639
           3       0.86      0.87      0.87       909
           4       0.91      0.92      0.92      2188
           5       0.92      0.89      0.91       113
           6       0.85      0.86      0.85       697
           7       0.88      0.89      0.88      1091
           8       0.88      0.90      0.89      1214
           9       0.83      0.79      0.81      1220
          10       0.84      0.85      0.85       725
          11       0.89      0.93      0.91       210
          12       0.88      0.88      0.88      2722
          13       0.87      0.84      0.85       341
          14       0.84      0.79      0.81       319
          15       0.00      0.00      0.00        13
          16       0.95      0.89      0.92        98
          17       0.80      0.72      0.76       248
          18       0.88    

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
